In [1]:
import pulp

lp_problem = pulp.LpProblem("Minimize_Total_Cost", pulp.LpMinimize)


generators = ["Type1", "Type2", "Type3"]
hydro = ["Type1", "Type2"]
periods = ["Period1", "Period2", "Period3", "Period4", "Period5"]


periods_hours = [6, 3, 6, 3, 6]
num_generators = {"Type1": 12, "Type2": 10, "Type3": 5}
costs_above_min = {"Type1": 2, "Type2": 1.30, "Type3": 3}
costs_min = {"Type1": 1000, "Type2": 2600, "Type3": 3000}
startup_costs = {"Type1": 2000, "Type2": 1000, "Type3": 500}
generator_limits = {"Type1": (850, 2000), "Type2": (1250, 1750), "Type3": (1500, 4000)}
load_demands = {"Period1": 15000, "Period2": 30000, "Period3": 25000, "Period4": 40000, "Period5": 27000}

hyOpLevel = {'Type1' : 900, 'Type2' : 1400}   # Li is the operating level of hydro i.
hyCostPerHour = {'Type1' : 90, 'Type2' : 150}
hyStartCost = {'Type1' : 1500, 'Type2' : 1200}
R = {'Type1' : 0.31, 'Type2' : 0.47}



n = pulp.LpVariable.dicts("Count", (generators, periods), lowBound=0, cat=pulp.LpInteger)
s = pulp.LpVariable.dicts("StartUp", (generators, periods), lowBound=0, cat=pulp.LpInteger)
x = pulp.LpVariable.dicts("ElectricityProduced", (generators, periods), lowBound=0, cat="Continuous")
h = pulp.LpVariable.dicts("Hydro", (hydro, periods), lowBound=0, cat=pulp.LpBinary)
t = pulp.LpVariable.dicts("HydroStart", (hydro, periods), lowBound=0, cat=pulp.LpBinary)
l = pulp.LpVariable.dicts("ReserviorHeight", (periods), lowBound=15, upBound=20, cat="Continuous")
p = pulp.LpVariable.dicts("PumpingCount", (periods), lowBound=0, cat="Continuous")





In [2]:
   
#Output must lie within the limits of the generators working:
for gen in generators:
    for per in periods:
        lp_problem += x[gen][per]>= n[gen][per] * generator_limits[gen][0] , f"mn_{gen}_{per}"
        lp_problem += x[gen][per]<= n[gen][per] * generator_limits[gen][1] , f"mx_{gen}_{per}"
#The extra guaranteed load requirement must be able to be met without starting up any more generators:
reserve_factor = 1.15
for per in periods:
    lp_problem += pulp.lpSum(n[gen][per] * generator_limits[gen][1] for gen in generators) +pulp.lpSum(hyOpLevel[gen] for gen in hydro) >= reserve_factor * load_demands[per], f"ReseverDemand_{per}"


lp_problem+= l[periods[0]] == 16, 'Midnight height'

for j in range(len(periods)):
    lp_problem += l[periods[j]] - l[periods[j-1]] - ((periods_hours[j]*p[periods[j]])/3000) + pulp.lpSum(h[gen][periods[j]]*periods_hours[j]*R[gen] for gen in hydro) == 0, f"ReserviorHeight_{periods[j]}"


for gen in hydro:
    for j in range(len(periods)):
        per = periods[j]
        prv_per = periods[(j-1)%len(periods)]
        #lp_problem += h[gen][per] >= t[gen][per] , f"hydrostart_{gen}_{per}"
        lp_problem += h[gen][per] - h[gen][prv_per]<= t[gen][per] , f"hydrostartdef_{gen}_{per}"




#The number of generators started in period j must equal the increase in number:
for gen in generators:
    for j in range(len(periods)):
        per = periods[j]
        prv_per = periods[(j-1)%len(periods)]
        lp_problem += s[gen][per] >= n[gen][per] - n[gen][prv_per] , f"xxx_{gen}_{per}"

#In addition all the integer variables have simple upper bounds corresponding to the total number of generators of each type.
for gen in generators:
    for j in range(len(periods)):
        lp_problem += n[gen][periods[j]] <= num_generators[gen], f"NumGenerators_{gen}_{periods[j]}"
        
for per in periods:
    lp_problem += pulp.lpSum((x[gen][per] for gen in generators)) + pulp.lpSum((h[gen][per]*hyOpLevel[gen] for gen in hydro)) - p[per] >= load_demands[per], f"LoadDemand_{per}"


#Objective Function (to be Minimized)


firstPart = pulp.lpSum(costs_above_min[gen] * periods_hours[periods.index(per)] * (x[gen][per] - generator_limits[gen][0] * n[gen][per]) for gen in generators for per in periods)
secondPart = pulp.lpSum(costs_min[gen] * n[gen][per] * periods_hours[periods.index(per)] for gen in generators for per in periods)
thirdPart = pulp.lpSum(startup_costs[gen] * s[gen][per] for gen in generators for per in periods)

fourthPart = pulp.lpSum(hyCostPerHour[gen]*periods_hours[periods.index(per)]*h[gen][per] for gen in hydro for per in periods)
fifthPart = pulp.lpSum(hyStartCost[gen]*t[gen][per] for gen in hydro for per in periods)

lp_problem += firstPart + secondPart + thirdPart + fourthPart + fifthPart, "TotalCost"
# Solve the problem
lp_problem.solve()


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/n5/1zvcrsg91l96m8bj6jfhq08h0000gn/T/f0d44825874e4aa5a6aac8b8c9c6a21b-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/n5/1zvcrsg91l96m8bj6jfhq08h0000gn/T/f0d44825874e4aa5a6aac8b8c9c6a21b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 91 COLUMNS
At line 478 RHS
At line 565 BOUNDS
At line 626 ENDATA
Problem MODEL has 86 rows, 75 columns and 221 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 984877 - 0.00 seconds
Cgl0003I 0 fixed, 15 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 70 rows, 74 columns (50 integer (20 of which binary)) and 203 elements
Cbc0038I Initial state - 7 integers unsatisfied sum - 

1

In [ ]:
lp_problem += h["Type1"][periods[0]]==0
lp_problem += h["Type2"][periods[0]]==0


lp_problem += h["Type1"][periods[1]]==0
lp_problem += h["Type2"][periods[1]]==0

lp_problem += h["Type1"][periods[2]]==0
lp_problem += h["Type2"][periods[2]]==0

lp_problem += h["Type1"][periods[3]]==0
lp_problem += h["Type2"][periods[3]]==1

lp_problem += h["Type1"][periods[4]]==0
lp_problem += h["Type2"][periods[4]]==1

lp_problem += x["Type1"][periods[0]]==10565
lp_problem += x["Type2"][periods[0]]==5250
lp_problem += x["Type3"][periods[0]]==0

lp_problem += x["Type1"][periods[1]]==14250
lp_problem += x["Type2"][periods[1]]==15750
lp_problem += x["Type3"][periods[1]]==0

lp_problem += x["Type1"][periods[2]]==10200
lp_problem += x["Type2"][periods[2]]==15750
lp_problem += x["Type3"][periods[2]]==0


lp_problem += x["Type1"][periods[3]]==21350
lp_problem += x["Type2"][periods[3]]==15750
lp_problem += x["Type3"][periods[3]]==1500


lp_problem += x["Type1"][periods[4]]==10200
lp_problem += x["Type2"][periods[4]]==15750
lp_problem += x["Type3"][periods[4]]==0

lp_problem += n["Type3"][periods[1]]==0

In [3]:
print("Status:", pulp.LpStatus[lp_problem.status])
print("Total Cost:", pulp.value(lp_problem.objective),"\n")

for per in periods:
    for gen in generators:
        print(f"x {per} - {gen}: {pulp.value(x[gen][per])}")
    print("")

for per in periods:
    for gen in generators:
        print(f"n {per} - {gen}: {pulp.value(n[gen][per])}")
    print("")


for per in periods:
    for gen in generators:
        print(f"s {per} - {gen} Start-Up: {pulp.value(s[gen][per])}")
    print("")


for per in periods:
    print(f"l {per} hydro level: {pulp.value(l[per])}")
    print("")


for per in periods:
    for gen in generators:
        print(f"n {per} - {gen} Count: {pulp.value(n[gen][per])}")
    print("")

for per in periods:
    for gen in hydro:
        print(f"t {per} - {gen} Start-Up: {pulp.value(t[gen][per])}")
    print("")

for per in periods:
    for gen in hydro:
        print(f"h {per} - {gen} Count: {pulp.value(h[gen][per])}")
    print("")

for per in periods:
    print(f"p {per} - Pumping: {pulp.value(p[per])}")
    print("")




Status: Optimal
Total Cost: 986630.0 

x Period1 - Type1: 10200.0
x Period1 - Type2: 5250.0
x Period1 - Type3: 0.0

x Period2 - Type1: 14250.0
x Period2 - Type2: 15750.0
x Period2 - Type3: 0.0

x Period3 - Type1: 10200.0
x Period3 - Type2: 15750.0
x Period3 - Type3: 0.0

x Period4 - Type1: 21350.0
x Period4 - Type2: 15750.0
x Period4 - Type3: 1500.0

x Period5 - Type1: 10565.0
x Period5 - Type2: 15750.0
x Period5 - Type3: 0.0

n Period1 - Type1: 12.0
n Period1 - Type2: 3.0
n Period1 - Type3: 0.0

n Period2 - Type1: 12.0
n Period2 - Type2: 9.0
n Period2 - Type3: 0.0

n Period3 - Type1: 12.0
n Period3 - Type2: 9.0
n Period3 - Type3: 0.0

n Period4 - Type1: 12.0
n Period4 - Type2: 9.0
n Period4 - Type3: 1.0

n Period5 - Type1: 12.0
n Period5 - Type2: 9.0
n Period5 - Type3: 0.0

s Period1 - Type1 Start-Up: 0.0
s Period1 - Type2 Start-Up: 0.0
s Period1 - Type3 Start-Up: 0.0

s Period2 - Type1 Start-Up: 0.0
s Period2 - Type2 Start-Up: 6.0
s Period2 - Type3 Start-Up: 0.0

s Period3 - Type1 St